# Arte A — Carga e Integración de Datasets
Este notebook implementa la preparación de datos (Pasos 1–15) y deja el dataset mensual listo para modelos.

In [ ]:
# PASO 1 — Importar librerías (instalación opcional)
# Si faltan paquetes, descomenta y ejecuta la instalación.
# !pip install pandas numpy matplotlib seaborn missingno saits

import pandas as pd
import numpy as np
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

## PASO 2 — Cargar información de estaciones (datos estáticos)
No se usa aún para el modelo, pero luego: altitud, lat/lon (features extras).

In [ ]:
# Asegúrate de que el notebook esté en el directorio raíz del proyecto.
stations_df = pd.read_csv("estaciones.csv")
stations_df.head()

## PASO 3 — Función genérica para cargar cada CSV de variable
Convierte cada archivo al formato largo: columnas `date | station | variable`.

In [ ]:
def load_variable_csv(path, variable_name):
    """
    Convierte un CSV por variable a formato largo:
    date | station | variable
    """
    df = pd.read_csv(path, parse_dates=["date"])

    df_long = df.melt(
        id_vars="date",
station",
,
,
,

: 
,
: { 
: 
 },
: [
4
,

: 
,
: { 
: 
 },
: [
PM2.5.csv", "PM2.5")
pm10 = load_variable_csv("PM10.csv", "PM10")
no2  = load_variable_csv("NO2.csv",  "NO2")
o3   = load_variable_csv("O3.csv",   "O3")
so2  = load_variable_csv("SO2.csv",  "SO2")
co   = load_variable_csv("CO.csv",   "CO")

tmp  = load_variable_csv("TMP.csv", "temperature")
hum  = load_variable_csv("HUM.csv", "humidity")
vel  = load_variable_csv("VEL.csv", "wind_velocity")
dirc = load_variable_csv("DIR.csv", "wind_direction")
llu  = load_variable_csv("LLU.csv", "precipitation")
pre  = load_variable_csv("PRE.csv", "pressure")
rs   = load_variable_csv("RS.csv",  "solar_radiation")

dfs = [pm25, pm10, no2, o3, so2, co, tmp, hum, vel, dirc, llu, pre, rs]
len(dfs)

## PASO 5 — Unir todas las variables en un solo DataFrame

In [ ]:
df = reduce(
    lambda left, right: pd.merge(
        left, right,
        on=["date", "station"],
        how="outer"
    ),
    dfs
 )
df.shape

## PASO 6 — Limpieza básica

In [ ]:
df.sort_values(["station", "date"], inplace=True)
df.reset_index(drop=True, inplace=True)

display(df.info())
df.head()

## PASO 7 — Convertir columnas a numéricas

In [ ]:
num_cols = df.columns.difference(["date", "station"])
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")
df[num_cols].dtypes

## PASO 8 — Visualizar valores faltantes (OBLIGATORIO)

In [ ]:
plt.figure(figsize=(15,5))
msno.matrix(df)
plt.title("Missing Data Across Variables and Stations")
plt.show()

## PASO 9 — Imputación con SAITS (por estación)
Nota: SAITS requiere el paquete `saits` y puede depender de PyTorch; la ejecución puede tardar.

In [ ]:
from saits.imputation import SAITS

num_cols = df.select_dtypes(include=np.number).columns
dfs_imputed = []

for station in df["station"].unique():
    temp = df[df["station"] == station].copy()
    # Config del modelo (parámetros básicos). Ajusta según necesidad.
    model = SAITS(n_steps=len(num_cols))
    imputed = model.fit_transform(temp[num_cols])
    temp[num_cols] = imputed
    dfs_imputed.append(temp)

df = pd.concat(dfs_imputed).sort_values(["station", "date"])
df.head()

# PARTE B — Preparación para Modelado
## PASO 10 — Resampling mensual (por estación/municipio)

In [ ]:
monthly = (
    df.set_index("date")
      .groupby("station")
      .resample("M")
      .agg({
          "PM2.5": "mean",
          "PM10": "mean",
          "NO2": "mean",
          "O3": "mean",
          "SO2": "mean",
          "CO": "mean",
          "temperature": "mean",
          "humidity": "mean",
          "wind_velocity": "mean",
          "precipitation": "sum",  # Lluvia = SUMA
          "pressure": "mean",
          "solar_radiation": "mean"
      })
      .reset_index()
)
monthly.head()

## PASO 11 — Feature engineering básico

In [ ]:
monthly["year"] = monthly["date"].dt.year
monthly["month"] = monthly["date"].dt.month
monthly[["date", "year", "month"]].head()

## PASO 12 — Lags de PM2.5

In [ ]:
for lag in [1, 3, 6, 12]:
    monthly[f"pm25_lag_{lag}"] = (
        monthly.groupby("station")["PM2.5"].shift(lag)
    )
monthly[["station", "date", "PM2.5", "pm25_lag_12"]].head(15)

## PASO 13 — Rolling statistics (promedio móvil)

In [ ]:
monthly["pm25_roll_3"] = (
    monthly.groupby("station")["PM2.5"]
    .rolling(3)
    .mean()
    .reset_index(0, drop=True)
)
monthly[["station", "date", "PM2.5", "pm25_roll_3"]].head(15)

## PASO 14 — Eliminar NaNs generados

In [ ]:
before = len(monthly)
monthly.dropna(inplace=True)
after = len(monthly)
print(f"Filas antes: {before}, después: {after}")
monthly.head()

## PASO 15 — Dataset FINAL listo para modelos

In [ ]:
display(monthly.head())
display(monthly.info())

### Guardado opcional del dataset mensual
Se puede guardar el dataset procesado en `outputs/monthly_dataset.csv`.

In [ ]:
import os
os.makedirs("outputs", exist_ok=True)
monthly.to_csv("outputs/monthly_dataset.csv", index=False)
"Archivo guardado en outputs/monthly_dataset.csv"

# EDA — Exploratory Data Analysis\nObjetivo: entender el comportamiento del PM2.5, su tendencia, estacionalidad y relación con variables meteorológicas.\n\nRequisitos previos: usamos el dataset `monthly` construido en los pasos anteriores (date | station | PM2.5 | temperature | humidity | wind_velocity | precipitation | ...).

In [ ]:
# 1️⃣ VISIÓN GENERAL DEL DATASET
print("Número de estaciones:", monthly["station"].nunique())
print("Rango de fechas:", monthly["date"].min(), "→", monthly["date"].max())
print("Observaciones totales:", len(monthly))

In [ ]:
# 2️⃣ TENDENCIA TEMPORAL DE PM2.5
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,4))
sns.lineplot(
    data=monthly,
    x="date",
    y="PM2.5",
    hue="station",
    legend=False
)
plt.title("Tendencia mensual de PM2.5 por estación")
plt.xlabel("Fecha")
plt.ylabel("PM2.5 (µg/m³)")
plt.show()

In [ ]:
# 3️⃣ PROMEDIO DE PM2.5 POR ESTACIÓN
station_avg = (
    monthly.groupby("station")["PM2.5"]
    .mean()
    .sort_values(ascending=False)
)

station_avg.plot(kind="bar", figsize=(10,4))
plt.title("PM2.5 promedio por estación")
plt.ylabel("PM2.5 (µg/m³)")
plt.show()

# MODELO ARIMA – PASO A PASO\nObjetivo: Modelar y pronosticar PM2.5 usando un modelo estadístico univariado basado en tendencia y estacionalidad.\n\nNota de dependencias: este bloque usa `statsmodels`. Si es necesario, instala con:\n\n```\n# !pip install statsmodels\n```

In [ ]:
# 0️⃣ PREPARACIÓN DE LA SERIE
station = "BELISARIO"

ts = (
    monthly[monthly["station"] == station]
    .set_index("date")["PM2.5"]
    .asfreq("MS")
 )

ts.head()

In [ ]:
# 1️⃣ VISUALIZAR LA SERIE
plt.figure(figsize=(10,4))
ts.plot()
plt.title(f"PM2.5 mensual – {station}")
plt.ylabel("PM2.5 (µg/m³)")
plt.show()

In [ ]:
# 2️⃣ PRUEBA DE ESTACIONARIEDAD (ADF)
from statsmodels.tsa.stattools import adfuller

adf_result = adfuller(ts.dropna())
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])

In [ ]:
# 3️⃣ DIFERENCIACIÓN (d) + ADF
ts_diff = ts.diff().dropna()

plt.figure(figsize=(10,4))
ts_diff.plot()
plt.title("Serie diferenciada (d=1)")
plt.show()

adf_diff = adfuller(ts_diff)
print("ADF Statistic (diff):", adf_diff[0])
print("p-value (diff):", adf_diff[1])

# MODELO LSTM – PASO A PASO
Objetivo: modelar PM2.5 con una red LSTM para capturar patrones no lineales y estacionales.

Nota de dependencias: requiere `tensorflow` y `scikit-learn`. Si hace falta:

# !pip install tensorflow scikit-learn

In [ ]:
# 1️⃣ LIBRERÍAS NECESARIAS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# 2️⃣ SELECCIONAR UNA ESTACIÓN (BELISARIO)
station = "BELISARIO"

df_lstm = (
    monthly[monthly["station"] == station]
    .set_index("date")[["PM2.5"]]
    .dropna()
 )

df_lstm.head()

In [ ]:
# 3️⃣ NORMALIZAR LOS DATOS
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df_lstm)
scaled[:5]

In [ ]:
# 4️⃣ CREAR SECUENCIAS (VENTANA 12)
def create_sequences(data, window=12):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled, 12)

print(X.shape, y.shape)

In [ ]:
# 5️⃣ TRAIN / TEST SPLIT (80/20)
split = int(len(X) * 0.8)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Asegurar forma correcta para LSTM: (samples, timesteps, features)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# 6️⃣ CONSTRUIR EL MODELO LSTM
model = Sequential([
    LSTM(50, activation="tanh", input_shape=(12, 1)),
    Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse"
)

model.summary()

In [ ]:
# 7️⃣ ENTRENAR EL MODELO
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    verbose=1
)

In [ ]:
# 8️⃣ PREDICCIÓN + INVERSE TRANSFORM
y_pred = model.predict(X_test)

# Invertir normalización
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv[:5], y_test_inv[:5]

In [ ]:
# 9️⃣ EVALUACIÓN (MAE)
mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv)
print("MAE LSTM:", mae_lstm)

In [ ]:
# 🔟 VISUALIZACIÓN
plt.figure(figsize=(10,4))
plt.plot(y_test_inv, label="Real")
plt.plot(y_pred_inv, label="Predicción LSTM")
plt.legend()
plt.title("LSTM – PM2.5")
plt.show()

In [ ]:
# 1️⃣1️⃣ PRONÓSTICO FUTURO (12 MESES)
last_window = scaled[-12:]
forecast = []

current = last_window

for _ in range(12):
    pred = model.predict(current.reshape(1,12,1))
    forecast.append(pred[0,0])
    current = np.vstack([current[1:], pred])

forecast = scaler.inverse_transform(np.array(forecast).reshape(-1,1))
forecast[:5]